<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=190/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image-prediction.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image-prediction.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Prediction Import
* This notebook walks you through the process of uploading model predictions to a Model Run. This notebook provides an example for each supported prediction type for image assets. 

A Model Run is a container for the predictions, annotations and metrics of a specific experiment in your ML model development cycle.

* For information on what types of predictions are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/upload-model-predictions#step-6-create-the-predictions-payload

* Notes:
    * If you are importing more than 1,000 mask predictions at a time, consider submitting separate jobs, as they can take longer than other prediction types to import.
    * After the execution of this notebook a complete Model Run with predictions will be created in your organization. 

# Installs

In [1]:
!pip install -q 'labelbox[data]'

     |████████████████████████████████| 184 kB 5.2 MB/s 
     |████████████████████████████████| 7.8 MB 58.9 MB/s 


# Imports

In [2]:
import labelbox
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox.schema.queue_mode import QueueMode
from labelbox import Client, LabelingFrontend, LabelImport, MediaType
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
import json
import uuid
import copy
import numpy as np
print(labelbox.__version__)

3.33.1


# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [3]:
# Add your api key as a string
API_KEY = ""
client = Client(api_key=API_KEY)

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Create a Model Run (for predictions) and a Project (for annotations)

We will be creating 
- a Model and a Model Run to contain model predictions
- a project to contain annotations

First, we create an ontology with all the possible tools and classifications supported for images. The official list of supported predictions and annotations that can be uploaded can be found:
- [predictions that can be uploaded to a Model Run](https://docs.labelbox.com/docs/upload-model-predictions#step-6-create-the-predictions-payload)
- [annotations that can be imported in a project as ground-truths](https://docs.labelbox.com/docs/import-ground-truth)

Note: the ontology of the Model Run does not need to match the ontology of the project. However, only the features present in the Model Run ontology can be uploaded as predictions and annotations to the Model Run.

In [6]:
ontology_builder = OntologyBuilder(
  tools=[ # List of Tool objects
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="box"), 
    Tool( # Polyline tool given the name "line"
      tool=Tool.Type.LINE, 
      name="line"), 
    Tool( # Point tool given the name "point"
      tool=Tool.Type.POINT, 
      name="point"), 
    Tool( # Polygon tool given the name "polygon"
      tool=Tool.Type.POLYGON, 
      name="polygon"), 
    Tool( # Segmentation mask tool given the name "mask"
      tool=Tool.Type.SEGMENTATION, 
      name="mask")], 
  classifications=[ # List of Classification objects
    Classification( # Text classification given the name "text"
      class_type=Classification.Type.TEXT,
      instructions="text"), 
    Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=Classification.Type.CHECKLIST, 
      instructions="checklist", 
      options=[
        Option(value="first_checklist_answer"),
        Option(value="second_checklist_answer")            
      ]
    ), 
    Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=Classification.Type.RADIO, 
      instructions="radio", 
      options=[
        Option(value="first_radio_answer"),
        Option(value="second_radio_answer")
      ]
    )
  ]
)

ontology = client.create_ontology("Ontology Image", ontology_builder.asdict())

We create a Model and a Model Run, to contain the predictions. 

In [7]:
# create Model
model = client.create_model(name="image_model_run", 
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

We create a project, to contain the annotations.

In [8]:
# Create a Labelbox project
project = client.create_project(name="image_project",                                    
                                    queue_mode=QueueMode.Batch,
                                    # Quality Settings setup 
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Image)
project.setup_editor(ontology)

### Create a dataset with a data row
We will upload predictions and annotations on this data row. 

In [9]:
# # Create one Labelbox dataset
dataset = client.create_dataset(name="image_prediction_import_demo_dataset")
# Grab an example image and create a Labelbox data row in the dataset
uploads = {
        "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
            # To learn more about Global Keys : https://docs.labelbox.com/docs/global-keys
        "global_key": "TEST-ID-%id" % uuid.uuid1()
    }
data_row = dataset.create_data_row(uploads)
print(data_row)

<DataRow {
    "created_at": "2022-12-22 00:35:51+00:00",
    "external_id": null,
    "global_key": "TEST-ID-199429380376703151620955766460100902924d",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
    "uid": "clbycrqin0kfb08wea4mp0jkb",
    "updated_at": "2022-12-22 00:35:51+00:00"
}>


### Send the data row to the Model Run and to the project

Get the data row IDs that we just uploaded

In [10]:
# Data row ID(s) to send to the Model Run and to the project.
datarow_ids = [dr.uid for dr in list(dataset.export_data_rows())]
print("datarow_ids: ",datarow_ids)

datarow_ids:  ['clbycrqin0kfb08wea4mp0jkb']


Send the data row to the Model Run

In [11]:
model_run.upsert_data_rows(datarow_ids)

True

Send the data row to the project

In [12]:
project.create_batch(
  "first-batch", # Each batch in a project must have a unique name
  datarow_ids, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

<Batch ID: 9b887b00-8190-11ed-9815-a565be1c194e>

### Create the predictions payload
We will upload it to the Model Run.


It is recommended to use the Python SDK's annotation types when importing labels into Labelbox.

Object predictions

In [13]:
# Confidence scores are optional.
# If no confidence is provided, 
# the prediction will be treated as if the confidence score equals 1

point_prediction=ObjectAnnotation(
    value=Point(x=850,y=150), # Coordinates for this point annotation
    name="point", # Name of the tool in your ontology
    confidence=0.5
)

box_prediction=ObjectAnnotation(
    value=Rectangle( # Coordinates for the top-left and bottom-right points of your bounding box, respectively
        start=Point(x=537,y=878),
        end=Point(x=832,y=1120)
    ),
    name="box", # Name of the tool in your ontology
    confidence=0.5
)

polyline_prediction=ObjectAnnotation(
    value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2514.353, y=229.471),Point(x=2409.492, y=152.092),Point(x=2254.322, y=201.962),Point(x=2204.491, y=140.463),Point(x=2116.123, y=254.716),
                Point(x=1752.247, y=133.949),Point(x=1753.838, y=34.438),Point(x=1539.772, y=32.61),Point(x=1543.442, y=107.552),
                Point(x=1438.869, y=124.903),Point(x=1408.941, y=308.149),Point(x=1143.128, y=370.815),Point(x=822.067, y=219.007),
                Point(x=782.367, y=319.216),Point(x=620.273, y=314.408),Point(x=573.114, y=238.16),Point(x=327.559, y=218.251),
                Point(x=318.087, y=125.064),Point(x=226.557, y=117.286),Point(x=216.648, y=235.61),Point(x=40.929, y=306.412)]
    ),
    name="line", # Name of the tool in your ontology
    confidence=0.5
)

polygon_prediction=ObjectAnnotation(
    value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1389.581,y=183.934),Point(x=2178.306,y=256.885),Point(x=2328.197,y=200.437),Point(x=2460.0,y=335.419),
                Point(x=2457.386,y=503.165),Point(x=2220.596,y=503.103),Point(x=2056.083, y=628.943),Point(x=2061.111,y=785.519),
                Point(x=1902.115, y=894.647),Point(x=1738.456,y=877.874),Point(x=1336.53,y=874.636),Point(x=1311.403,y=758.579),
                Point(x=1253.853,y=751.74),Point(x=1245.264, y=453.461),Point(x=1326.011,y=421.129)]
    ),
    name="polygon", # Name of the tool in your ontology
    confidence=0.5
)

mask_prediction=ObjectAnnotation(
    value=Mask( # Numpy array representation of a segmentation mask with the corresponding values within the array that represent the segmentation mask
        mask=MaskData(
            arr=np.zeros([300,350,3],dtype='uint8') # Creating an example numpy array to represent a mask creating a square from pixels 0,0 to 128,128
        ),
        color=(0,0,0) # Identifying what values in the numpy array correspond to the mask annotation (since I made a 3-D array with all zeroes, here it's 0,0,0)
    ),
    name="mask", # Name of the tool in your ontology
    confidence=0.5
)

Classification predictions

In [14]:
# Confidence scores are optional.
# If no confidence is provided, 
# the prediction will be treated as if the confidence score equals 1

checklist_prediction=ClassificationAnnotation(
    value=Checklist(
        answer=[ # List of the checklist answers in your ontology
            ClassificationAnswer(
                name="first_checklist_answer",
                confidence=0.5
            ),
            ClassificationAnswer(
                name="second_checklist_answer",
                confidence=0.5
            )
        ]
    ), 
    name="checklist" # Name of the classification in your ontology
)

radio_prediction=ClassificationAnnotation(
    value=Radio(
        answer=ClassificationAnswer(
            name="first_radio_answer", # Name of the radio answer in your ontology
            confidence=0.5
        )
    ), 
    name="radio" # Name of the classification in your ontology
)

# Confidence is not supported for text prediction
text_prediction=ClassificationAnnotation(
    value=Text( # String value for the text annotation
        answer="the answer to the text question",
    ), 
    name="text" # Name of the classification in your ontology
)


Create a Label object with all of the predictions created previously.

In [15]:
# Create a Label object by identifying the applicavle data row in Labelbox and providing a list of annotations
label_prediction = Label(
    data=ImageData(
        uid=data_row.uid),
    annotations = [
        point_prediction, box_prediction, text_prediction, 
        checklist_prediction, radio_prediction,
        polygon_prediction, mask_prediction, polyline_prediction, 

    ]
)

# Create urls to mask data for upload
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url

label_prediction.add_url_to_masks(signing_function)

label_prediction.__dict__

{'uid': None,
 'data': ImageData(im_bytes=None,file_path=None,url=None,arr=None),
 'annotations': [ObjectAnnotation(confidence=0.5, name='point', feature_schema_id=None, extra={}, value=Point(extra={}, x=850.0, y=150.0), classifications=[]),
  ObjectAnnotation(confidence=0.5, name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=537.0, y=878.0), end=Point(extra={}, x=832.0, y=1120.0)), classifications=[]),
  ClassificationAnnotation(name='text', feature_schema_id=None, extra={}, value=Text(answer='the answer to the text question')),
  ClassificationAnnotation(name='checklist', feature_schema_id=None, extra={}, value=Checklist(name='checklist', answer=[ClassificationAnswer(confidence=0.5, name='first_checklist_answer', feature_schema_id=None, extra={}, keyframe=None), ClassificationAnswer(confidence=0.5, name='second_checklist_answer', feature_schema_id=None, extra={}, keyframe=None)])),
  ClassificationAnnotation(name='radio', feature_schema_id

### Create the annotations payload
We will upload it to the project.

It is recommended to use the Python SDK's annotation types when importing labels into Labelbox.

Object annotations

In [16]:
point_annotation=ObjectAnnotation(
    value=Point(x=882,y=159), # Coordinates for this point annotation
    name="point" # Name of the tool in your ontology
)

box_annotation=ObjectAnnotation(
    value=Rectangle( # Coordinates for the top-left and bottom-right points of your bounding box, respectively
        start=Point(x=557,y=898),
        end=Point(x=852,y=1140)
    ),
    name="box" # Name of the tool in your ontology
)

polyline_annotation=ObjectAnnotation(
    value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2534.353, y=249.471),Point(x=2429.492, y=182.092),Point(x=2294.322, y=221.962),Point(x=2224.491, y=180.463),Point(x=2136.123, y=204.716),
                Point(x=1712.247, y=173.949),Point(x=1703.838, y=84.438),Point(x=1579.772, y=82.61),Point(x=1583.442, y=167.552),
                Point(x=1478.869, y=164.903),Point(x=1418.941, y=318.149),Point(x=1243.128, y=400.815),Point(x=1022.067, y=319.007),
                Point(x=892.367, y=379.216),Point(x=670.273, y=364.408),Point(x=613.114, y=288.16),Point(x=377.559, y=238.251),
                Point(x=368.087, y=185.064),Point(x=246.557, y=167.286),Point(x=236.648, y=285.61),Point(x=90.929, y=326.412)]
    ),
    name="line" # Name of the tool in your ontology
)

polygon_annotation=ObjectAnnotation(
    value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1489.581,y=183.934),Point(x=2278.306,y=256.885),Point(x=2428.197,y=200.437),Point(x=2560.0,y=335.419),
                Point(x=2557.386,y=503.165),Point(x=2320.596,y=503.103),Point(x=2156.083, y=628.943),Point(x=2161.111,y=785.519),
                Point(x=2002.115, y=894.647),Point(x=1838.456,y=877.874),Point(x=1436.53,y=874.636),Point(x=1411.403,y=758.579),
                Point(x=1353.853,y=751.74),Point(x=1345.264, y=453.461),Point(x=1426.011,y=421.129)]
    ),
    name="polygon" # Name of the tool in your ontology
)

mask_annotation=ObjectAnnotation(
    value=Mask( # Numpy array representation of a segmentation mask with the corresponding values within the array that represent the segmentation mask
        mask=MaskData(
            arr=np.zeros([400,450,3],dtype='uint8') # Creating an example numpy array to represent a mask creating a square from pixels 0,0 to 128,128
        ),
        color=(0,0,0) # Identifying what values in the numpy array correspond to the mask annotation (since I made a 3-D array with all zeroes, here it's 0,0,0)
    ),
    name="mask" # Name of the tool in your ontology
)

Classification annotations

In [17]:
text_annotation=ClassificationAnnotation(
    value=Text( # String value for the text annotation
        answer="the answer to the text question" 
    ), 
    name="text" # Name of the classification in your ontology
)

checklist_annotation=ClassificationAnnotation(
    value=Checklist(
        answer=[ # List of the checklist answers in your ontology
            ClassificationAnswer(name="first_checklist_answer"),
            ClassificationAnswer(name="second_checklist_answer")
        ]
    ), 
    name="checklist" # Name of the classification in your ontology
)

radio_annotation=ClassificationAnnotation(
    value=Radio(
        answer=ClassificationAnswer(
            name="second_radio_answer" # Name of the radio answer in your ontology
        )
    ), 
    name="radio" # Name of the classification in your ontology
)

Create a Label object with all of the annotations created previously.

In [18]:
# Create a Label object by identifying the applicavle data row in Labelbox and providing a list of annotations
label_annotation = Label(
    data=ImageData(
        uid=data_row.uid),
    annotations = [
        point_annotation, box_annotation, polyline_annotation, polygon_annotation, mask_annotation,
        text_annotation, checklist_annotation, radio_annotation
    ]
)

# Create urls to mask data for upload
def signing_function(obj_bytes: bytes) -> str:
    url = client.upload_data(content=obj_bytes, sign=True)
    return url

label_annotation.add_url_to_masks(signing_function)

label_annotation.__dict__

{'uid': None,
 'data': ImageData(im_bytes=None,file_path=None,url=None,arr=None),
 'annotations': [ObjectAnnotation(confidence=None, name='point', feature_schema_id=None, extra={}, value=Point(extra={}, x=882.0, y=159.0), classifications=[]),
  ObjectAnnotation(confidence=None, name='box', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=557.0, y=898.0), end=Point(extra={}, x=852.0, y=1140.0)), classifications=[]),
  ObjectAnnotation(confidence=None, name='line', feature_schema_id=None, extra={}, value=Line(extra={}, points=[Point(extra={}, x=2534.353, y=249.471), Point(extra={}, x=2429.492, y=182.092), Point(extra={}, x=2294.322, y=221.962), Point(extra={}, x=2224.491, y=180.463), Point(extra={}, x=2136.123, y=204.716), Point(extra={}, x=1712.247, y=173.949), Point(extra={}, x=1703.838, y=84.438), Point(extra={}, x=1579.772, y=82.61), Point(extra={}, x=1583.442, y=167.552), Point(extra={}, x=1478.869, y=164.903), Point(extra={}, x=1418.941, y=318.149

### Upload the annotations payload to the project

In [19]:
## Create a label list 
label_list_annotation = [label_annotation]

# Convert the annotation label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
ndjson_annotation = list(NDJsonConverter.serialize(label_list_annotation))

# Upload the annotation label to the project using Label Import
upload_job_annotation = LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="annotation_import_job",
    labels=ndjson_annotation)

# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)

Errors: []


### Send the annotations to the Model Run

Get the label IDs that we just uploaded


In [20]:
# get the labels id from the project
label_ids = [x['ID'] for x in project.export_labels(download=True)]
print("label_ids: ",label_ids)

label_ids:  ['clbycrz8s000l0g3ejhyi3w4g']


In [21]:
model_run.upsert_labels(label_ids)

True

### Upload the predictions payload to the Model Run

In [22]:
## Create a label list 
label_list_prediction = [label_prediction]

# Convert the prediction label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
ndjson_prediction = list(NDJsonConverter.serialize(label_list_prediction))

# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=ndjson_prediction)

# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_prediction.errors)


Errors: []


## Cleanup 

In [23]:
# mal_project.delete()
# li_project.delete()
# dataset.delete()